In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import sys
sys.path.insert(0, "../")

In [52]:
import kangaroo

In [53]:
import pandas as pd

In [54]:
pd_data = pd.read_csv("../data/country columns.csv")[["country of birth father", "country of birth mother", "country of birth self"]]

In [55]:
from kangaroo.feature_exploration import compute_chi2_matrix

In [56]:
pd_data.describe()

,country of birth father,country of birth mother,country of birth self
count,199523,199523,199523
unique,43,43,43
top,United-States,United-States,United-States
freq,159163,160479,176989


In [57]:
import numpy as np
import pandas as pd
from scipy.stats import distributions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder

def compute_chi2_matrix(pd_data: pd.DataFrame) -> np.array:
    """Computes a matrix with chi2 tests for each pair of variables.
    When the score for a pair is 0, it means they are highly related. Where it is 1, they are not related.
    :param pd_data: dataframe with N categorical columns
    :return: numpy matrix of size N x N
    """
    dict_unique_values = {}
    for c in pd_data.columns:
        le = LabelEncoder()
        le.fit(pd_data[c])
        pd_data[c] = le.transform(pd_data[c])
        dict_unique_values[c] = len(le.classes_)

    result_tests = np.zeros(shape=(len(pd_data.columns), len(pd_data.columns)))
    for i, c in enumerate(pd_data.columns):
        fs = SelectKBest(score_func=chi2, k='all')
        _ = fs.fit_transform(pd_data.values, pd_data[c])
        degrees_of_freedom = [(dict_unique_values[c] - 1) * (dict_unique_values[x] - 1) for x in pd_data.columns]
        result_tests[i, :] = fs.scores_
    return result_tests


In [58]:
x = compute_chi2_matrix(pd_data[["country of birth father", "country of birth self"]]) 
1- x / np.max(x)

/home/rodri/pruebaZeta/pruebaZeta/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


array([[0.        , 0.75016947],
       [0.58002799, 0.43551656]])

In [59]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
pd_data["country of birth self index"] = le.fit_transform(pd_data["country of birth self"])
pd_data["country of birth father index"] = le.transform(pd_data["country of birth father"])

In [60]:
from sklearn import tree
t = tree.DecisionTreeClassifier()


In [78]:
from sklearn.metrics import accuracy_score
def compute_distance_prediction(c, d):
    le = preprocessing.LabelEncoder()
    le.fit(pd_data[c])
    t = tree.DecisionTreeClassifier()
    t.fit([le.transform(pd_data[c])], [le.transform(pd_data[d])])
    x = t.predict([le.transform(pd_data[[c]])])
    r = le.inverse_transform(x[0, :])
    return accuracy_score(pd_data[d], r)
print(compute_distance_prediction("country of birth self", "country of birth father"))
print(compute_distance_prediction("country of birth self", "country of birth self"))

/home/rodri/pruebaZeta/pruebaZeta/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[' United-States' ' United-States' ' Vietnam' ... ' Poland'
 ' United-States' ' ?']
1.0


/home/rodri/pruebaZeta/pruebaZeta/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[' United-States' ' United-States' ' Vietnam' ... ' Germany'
 ' United-States' ' ?']
1.0


In [21]:
data_crosstab = pd.crosstab(pd_data['country of birth father'],
                            pd_data['country of birth self'])
print(data_crosstab)


country of birth self             ?   Cambodia   Canada   China   Columbia  \
country of birth father                                                      
 ?                             2907          0       53       9          2   
 Cambodia                        11         84        0       1          0   
 Canada                           5          0      433       0          0   
 China                           23          9        1     458          0   
 Columbia                        11          0        0       0        414   
 Cuba                             4          0        0       0          1   
 Dominican-Republic               3          0        0       0          0   
 Ecuador                          5          0        1       0          0   
 El-Salvador                      4          0        1       0          0   
 England                          4          0       42       0          0   
 France                           8          0        3       0 